In [1]:
## Package
import glob
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from typing import List
from lxml import etree
from preTraitements.xml import get_X_Y_from_root
from preTraitements.xml import get_tree_root_from_file
from sklearn.preprocessing import FunctionTransformer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.preprocessing import StandardScaler
import os

In [2]:
import re
vector = {}
pattern = re.compile(r"(\w+).+")
with open("vectors.txt",'r',encoding="utf-8") as file:
    line = file.readline()
    while line:
        m=re.match(pattern,line)
        #print(line[len(m.group(1)):].split())
        if m:
            vector[m.group(1)]=line[len(m.group(1)):].split()
        line = file.readline()


In [3]:
tree_train, root_train = get_tree_root_from_file("./corpus/train_deft09_parlement_appr.xml/deft09_parlement_appr_fr.xml")
X_train, y_train = get_X_Y_from_root(root_train)

tree_test, root_test = get_tree_root_from_file("./corpus/deft09_parlement_test.xml/deft09_parlement_test_fr.xml")
X_test, y_test = get_X_Y_from_root(root_test)

In [276]:
X_train_sample = X_train[:1]
y_train_sample = y_train[:1]

In [5]:
# Global parameters
#root folder
root_folder='.'
glove_filename='vectors.txt'

# Variable for data directory
glove_path = os.path.abspath(glove_filename)


In [6]:
# We just need to run this code once, the function glove2word2vec saves the Glove embeddings in the word2vec format 
# that will be loaded in the next section
from gensim.scripts.glove2word2vec import glove2word2vec
glove_filename='vectors.txt'
#glove_input_file = glove_filename
word2vec_output_file = glove_filename+'.word2vec'
glove2word2vec(glove_path, word2vec_output_file)

/tmp/ipykernel_221/3652708010.py:7: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  glove2word2vec(glove_path, word2vec_output_file)


(41153, 50)

In [7]:
from gensim.models import KeyedVectors
word2vec_output_file = glove_filename+'.word2vec'
glove = KeyedVectors.load_word2vec_format(word2vec_output_file, binary=False)

In [167]:
#Show a word embedding
print('Madame: ',glove.get_vector('madame'))

Madame:  [ 0.013859  0.451532 -0.487101  0.168849 -0.327894  0.146996 -0.224299
 -0.178038 -0.070914 -0.51272  -0.421458 -0.469794 -0.545544  0.139949
 -0.006503 -0.140751  0.333008 -0.048612  0.200209  0.376872  0.138722
 -0.271306  0.140596  0.33297   0.664384  0.201292 -0.005523 -0.746854
 -0.041821 -0.062128 -0.132737 -0.275511  0.495469  0.177609 -0.082183
 -0.180198  0.092252  0.044848  0.353179  0.449488 -0.322928 -0.258066
 -0.727073  0.10923   0.008607  0.232664  0.044286  0.068772  0.311111
  0.15182 ]


In [293]:
#remplace un CountVectorizer()
def GloveVectorizer(X_train):
    model = glove
    n=0
    m = glove.get_vector('président')
    #docs = {}
    docs = np.zeros((len(X_train),m.shape[0]))
    for document in X_train:
        word_vectors=[]
        for token in document.split():
            if token in model:
                word_vectors.append(model[token])
        if len(word_vectors)>0:
            word_vectors = np.array(word_vectors)
            print(np.array(word_vectors))
            docs[n]=word_vectors.mean(axis=0)
            print(docs[n])
        n=n+1
    return docs

In [294]:
X_vect = GloveVectorizer(X_train_sample)

[[-0.531231 -0.708816 -0.114237 ...  0.576635 -0.930365 -0.731844]
 [-0.320647 -0.258935 -0.583948 ...  0.525978 -1.136862 -0.266057]
 [-0.61792  -0.193781 -0.248724 ...  0.322474 -0.845926 -0.831415]
 ...
 [-0.141048  0.359355 -0.612708 ... -0.374679  0.125947  0.22027 ]
 [ 0.053538  0.243089  0.213892 ...  0.069623 -1.446567  0.070551]
 [-0.571903 -0.12516  -1.228226 ...  1.142751 -1.138536 -0.755584]]
[ 0.01528353 -0.17387539 -0.04330047  0.15780458  0.62494183  0.64479375
  0.00526036 -0.63278937  0.7071436   0.44991535  0.34993812 -0.03328431
  0.18640304  0.07692374 -0.00927639 -0.27934754 -0.71318609 -0.15461557
 -0.07641554  0.27266151 -0.13817562  0.813187   -0.42138663  0.09191747
 -0.58466172  0.03370244 -0.11750077  0.78793919 -0.31851864 -0.28415084
  0.24724312  1.36462593  0.07918341 -0.42685422  0.11637811 -0.16698672
  0.03501559 -0.02045711  0.22413215 -0.50469184  0.42869824  0.2786687
  0.15111926  0.06130914 -0.34547037  0.3875843  -0.00580706  0.27416426
 -0.94076

In [298]:
X_vect[0][0]

0.015283526852726936

In [242]:
X_vect.shape

(500, 50)

In [264]:
v = GloveVectorizer(X_train_sample)

In [265]:
clf = LinearSVC()
mod = clf.fit(X_vect,y_train_sample)

In [266]:
test_x_ok = GloveVectorizer(X_test)

In [267]:
y_pred = mod.predict(test_x_ok)

In [271]:
print(clf.score(X_vect,y_train_sample))

0.448


In [272]:
###pipeline ??? 
from sklearn.feature_extraction import DictVectorizer
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.svm import LinearSVC, SVC
LinearSVC_classifier = SklearnClassifier(SVC(kernel='linear',probability=True))
svm = SVC(
        kernel="linear",probability=True
    )

pipeline = Pipeline([
    ('glove_vect',FunctionTransformer(GloveVectorizer)),
    #('vect',DictVectorizer()),
    ('clf',svm)
])

pipeline

Pipeline(steps=[('glove_vect',
                 FunctionTransformer(func=<function GloveVectorizer at 0x7ffa88cb2b80>)),
                ('clf', SVC(kernel='linear', probability=True))])

In [273]:
pipeline.fit(X_train_sample,y_train_sample)

Pipeline(steps=[('glove_vect',
                 FunctionTransformer(func=<function GloveVectorizer at 0x7ffa88cb2b80>)),
                ('clf', SVC(kernel='linear', probability=True))])

In [274]:
pipeline.score(X_train_sample,y_train_sample)

0.362

In [275]:
pipeline.predict_proba(X_test)

array([[0.12481099, 0.17754994, 0.32707494, 0.24531349, 0.12525065],
       [0.1178912 , 0.15422346, 0.34736599, 0.25853471, 0.12198465],
       [0.10280593, 0.10372673, 0.36916321, 0.29718421, 0.12711993],
       ...,
       [0.0885924 , 0.10973989, 0.36596483, 0.28451211, 0.15119076],
       [0.12174019, 0.15426538, 0.35806682, 0.27087265, 0.09505496],
       [0.13070995, 0.13297865, 0.32790209, 0.28457089, 0.12383842]])